1. Annotations are redundant, it means that a protein may be annoteated with a term as well as its coarse grained terms.
2. All GO terms in quickgo are not alt ids in go-basic.obo

## 存在的问题：
1. 蛋白质数量还比较少

In [2]:
import os
import re
import gzip
import random
import pickle
import tarfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import chain
from collections import defaultdict

from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB.Polypeptide import PPBuilder

In [3]:
# organisms = ["arabidopsis", "fly", "mouse", "worm", "yeast", "human"]
# taxons = ["3702", "7227", "10090", "6239", "559292", "9606"]
# organisms = ["fly", "mouse", "worm", "yeast", "human"]
# taxons = ["7227", "10090", "6239", "559292", "9606"]
seed = 99

#! whether Use microbiology 
organisms = ["human"]
taxons = ["9606"]

#! params
processed_go_dir = "./processed/gograph/"
quickgo_dir = "./raw/quickgo"
uniprot_dir = "./raw/uniprot"
uniref_cluster_fp = f"./raw/uniref/cluster_member_six_module_orgas_50.tsv.gz"
alphafold_dir = "./raw/alphafold"
out_dir = "./processed/quickgo"

# evidence code that are excluded
excluded_evidence = ["ISS", "ISO", "ISA", "ISM", "IGC", "RCA", "IEA"]

min_pro_per_term = 30   # the minimum proteins num of each term
min_term_depth = {"bp":5 , "cc":4 , "mf":4}      # the minimum depth of reserved terms

max_seq_len = 800
min_seq_len = 50
non_canonical_aa = re.compile("[UZOBJX]")
cluster_max_pro_num = 1     # number of proteins in each cluster

In [4]:
class Annotation:
    def __init__(self, db, dbid, goid, evidence, aspect, taxon, organism):
        self.db = db
        self.dbid = dbid
        self.goid = goid
        self.evidence = evidence
        self.aspect = aspect
        self.taxon = taxon
        self.organism = organism

    def __str__(self):
        string = f"ID: {self.dbid}, GO: {self.goid}, Evidence: {self.evidence}, Aspect: {self.aspect}, Taxon: {self.taxon}"
        return string


class Protein:
    def __init__(self, dbid, orga):
        self.dbid = dbid
        self.annots = []
        self.orga = orga
        self.sequence = None
        self.stru = None
        self.ca_coords = None
    
    def add_annot(self, annot):
        assert annot.dbid == self.dbid
        for _annot in self.annots:
            #! Note, there may be same GO terms of different sources or evidence for a protein
            if annot.goid == _annot.goid:
                return
        self.annots.append(annot)
    
    def set_sequence(self, seq):
        self.sequence = seq

    def set_ca_coords(self, ca_coords):
        assert len(self.sequence) == ca_coords.shape[0]
        self.ca_coords = ca_coords

    def __str__(self):
        string = f"{self.dbid}, {' '.join([a.goid for a in self.annots])}, {self.sequence}"
        return string

### Read Raw GAF Files

In [4]:
def read_zipped_gaf(orga):
    fp = os.path.join(quickgo_dir, f"goa_{orga}.gaf.gz")
    with gzip.open(fp, "rb") as f:
        lines = [l.decode() for l in f.readlines()]
        lines = [l for l in lines if not l.startswith("!")]
    
    orga_annotations = []
    for line in lines:
        items = line.split("\t")
        assert items[11] == 'protein'
        annot = Annotation(db=items[0], dbid=items[1], goid=items[4], \
            evidence=items[6], aspect=items[8], taxon=items[12], organism=orga)
        orga_annotations.append(annot)
    return orga_annotations

annotations = {}
for orga in organisms:
    annotations[orga] = read_zipped_gaf(orga)

### Remove Annotation of Other Taxons

In [5]:
for orga, taxon in zip(organisms, taxons):
    valid_annots = [a for a in annotations[orga] if a.taxon == "taxon:" + taxon]

    pre_annots_num = len(annotations[orga])
    cur_annots_num = len(valid_annots)

    annotations[orga] = valid_annots
    print(f"{cur_annots_num} of {pre_annots_num} ({round(cur_annots_num/pre_annots_num*100, 2)}%) annotations in {orga} are reserved with {taxon}")

635148 of 635716 (99.91%) annotations in human are reserved with 9606


### Step 1: Remove Annotation with excluded evidence codes

In [6]:
for orga in organisms:
    valid_annots = [a for a in annotations[orga] if a.evidence not in excluded_evidence]

    pre_annots_num = len(annotations[orga])
    cur_annots_num = len(valid_annots)

    annotations[orga] = valid_annots
    print(f"{cur_annots_num} of {pre_annots_num} ({round(cur_annots_num/pre_annots_num*100, 2)}%) annotations in {orga} are reserved without IEA")

528969 of 635148 (83.28%) annotations in human are reserved without IEA


### Gather Annotations into Proteins

In [7]:
"""
In Annotations, a protein may have several same GO terms of
different source or evidence.
"""
proteins = defaultdict(dict)
for orga in organisms:
    for annot in annotations[orga]:
        proid = annot.dbid
        if proid not in proteins[orga]:
            proteins[orga][proid] = Protein(proid, orga)
        proteins[orga][proid].add_annot(annot)

    annots_num = sum([len(pro.annots) for pro in proteins[orga].values()])
    protein_num = len(proteins[orga])
    print(f"In {orga}, {protein_num} proteins, {annots_num} annotaions ({round(annots_num/protein_num, 2)} annots per protein).")

In human, 18850 proteins, 199682 annotaions (10.59 annots per protein).


### Collect Protein Sequences from Uniprot

In [8]:
"""read fasta files"""
for orga, taxon in zip(organisms, taxons):    
    with gzip.open(os.path.join(uniprot_dir, f"uniprot-{taxon}-{orga}.fasta.gz"), "rb") as f:
        lines = [l.decode() for l in f.readlines()]

    cur_id = None
    for l in lines:
        if l[0] == ">":
            _id = l.rstrip().split("|")[1]
            if cur_id is not None:
                assert len(line_buffer) != 0
                seq = re.sub("\s", "", "".join(line_buffer))
                if cur_id in proteins[orga]:
                    proteins[orga][cur_id].set_sequence(seq)

            cur_id = _id
            line_buffer = []
        else:
            line_buffer.append(l)
    if len(line_buffer) != 0:
        seq = re.sub("\s", "", "".join(line_buffer))
        if cur_id in proteins[orga]:
            proteins[orga][cur_id].set_sequence(seq)
        line_buffer = []

    pre_pro_num = len(proteins[orga])
    valid_pros = {dbid: pro for dbid, pro in proteins[orga].items() if pro.sequence is not None}
    proteins[orga] = valid_pros
    cur_pro_num = len(valid_pros)

    print(f"In {orga} has {cur_pro_num} of {pre_pro_num} ({round(cur_pro_num/pre_pro_num*100, 2)}%) has protein sequences in UniProt.")

In human has 18850 of 18850 (100.0%) has protein sequences in UniProt.


### Step 2: Remove Proteins of Too Long or Too Short

In [9]:
"""Filter sequences"""
def is_length_valid(pro):
    l = len(pro.sequence)
    return min_seq_len <= l and l <= max_seq_len

for orga in organisms:
    pros = {dbid: p for dbid, p in proteins[orga].items() if is_length_valid(p)}

    pre_protein_num = len(proteins[orga])
    cur_protein_num = len(pros)
    pre_annots_num = sum([len(p.annots) for p in proteins[orga].values()])
    cur_annots_num = sum([len(p.annots) for p in pros.values()])
    print(f"In {orga}, {cur_protein_num} of {pre_protein_num} ({round(cur_protein_num / pre_protein_num * 100)}%) proteins reserved,",
        f"{cur_annots_num} of {pre_annots_num} ({round(cur_protein_num / pre_protein_num * 100)}%) annotaions reserved",
        f"({round(cur_annots_num/cur_protein_num, 2)} annots per protein).")
    
    proteins[orga] = pros

In human, 15260 of 18850 (81%) proteins reserved, 154760 of 199682 (81%) annotaions reserved (10.14 annots per protein).


### Step 3: Remove Proteins with Non-canonical Amino Acids

In [10]:
def is_sequence_valid(pro):
    if re.search(non_canonical_aa, pro.sequence) is not None:
        return False
    else:
        return True

for orga in organisms:
    pros = {dbid: p for dbid, p in proteins[orga].items() if is_sequence_valid(p)}

    pre_protein_num = len(proteins[orga])
    cur_protein_num = len(pros)
    pre_annots_num = sum([len(p.annots) for p in proteins[orga].values()])
    cur_annots_num = sum([len(p.annots) for p in pros.values()])
    print(f"In {orga}, {cur_protein_num} of {pre_protein_num} ({round(cur_protein_num / pre_protein_num * 100)}%) proteins reserved,",
        f"{cur_annots_num} of {pre_annots_num} ({round(cur_protein_num / pre_protein_num * 100)}%) annotaions reserved",
        f"({round(cur_annots_num/cur_protein_num, 2)} annots per protein).")
    
    proteins[orga] = pros

In human, 15233 of 15260 (100%) proteins reserved, 154552 of 154760 (100%) annotaions reserved (10.15 annots per protein).


### Collect PDBs from AlphaFold

In [11]:
"""Align alphafold files with organisms"""
orga_afdb_fn = {}
for fn in os.listdir(alphafold_dir):
    fn_taxon = fn.split("_")[1]
    if fn_taxon in taxons:
        orga = organisms[taxons.index(fn_taxon)]
    else:
        continue
    orga_afdb_fn[orga] = fn

In [12]:
"""Read tar file"""

io = PDBIO()
parser = PDBParser()
builder = PPBuilder()
for orga in organisms:
    fp = os.path.join(alphafold_dir, orga_afdb_fn[orga])

    name_mem_dict = {}
    tar = tarfile.open(fp, "r")
    for tarmem in tar.getmembers():
        tarname = tarmem.name
        if "cif.gz" in tarname:
            continue
        dbid = tarname.split("-")[1]
        name_mem_dict[dbid] = tarmem

    pros = {}
    no_pdbs = []
    for dbid, pro in tqdm(proteins[orga].items()):
        if dbid not in name_mem_dict:
            no_pdbs.append(dbid)
            continue
        handler = gzip.open(tar.extractfile(name_mem_dict[dbid]), "rt")
        
        stru = parser.get_structure(dbid, handler)
        assert len(stru) == 1   # only one model
        assert len(stru[0]) == 1    # only one chain
        
        io.set_structure(stru[0]['A'])
        io.save(os.path.join(out_dir, "pdbs", dbid+".pdb"))

        pp = builder.build_peptides(stru[0]['A'])[0]
        sequence = pp.get_sequence()
        # ca_coords = np.array([ca.coord for ca in pp.get_ca_list()])
        try:
            assert sequence == pro.sequence
        except AssertionError:
            no_pdbs.append(dbid)
            continue

        # pro.set_ca_coords(ca_coords)
        pros[dbid] = pro

    proteins[orga] = pros
    print(f"In {orga}, {len(pros)} proteins reserved, {len(no_pdbs)} protein(s) has no pdb file in alphafold {orga} dataset.")

100%|██████████| 15233/15233 [39:45<00:00,  6.38it/s] 

In human, 15069 proteins reserved, 164 protein(s) has no pdb file in alphafold human dataset.


### Turn Protein Dict into List

In [13]:
"""Turn proteins into list"""
for orga in proteins:
    proteins[orga] = proteins[orga].values()
proteins = list(chain(*proteins.values()))

### Step 4: Filter Protein with Uniref50. Only one member is kept in each cluster.

In [14]:
"""Parse UniRef clusters"""
pro_clusters = {}

uniref = gzip.open(uniref_cluster_fp, "rb")
df = pd.read_csv(uniref, delimiter="\t")

for idx, row in df.iterrows():
    entry = row["Entry"]
    member_string = row["Members"]

    for pro_taxon in member_string.strip().split(";"):
        accessions, _ = pro_taxon.strip().split(":")
        for _access in accessions.strip().split(","):
            pro_clusters[_access] = entry

In [15]:
""" Gather proteins into clusters"""
cluster_proteins = defaultdict(list)
for pro in proteins:
    try:
        clusterid = pro_clusters[pro.dbid]
    except:
        print(f"{pro.dbid} not exists in Uniref clusters of {orga}!")
        continue
    cluster_proteins[clusterid].append(pro)

cluster_num = len(cluster_proteins)
protein_num = sum([len(cluster) for cluster in cluster_proteins.values()])
print(f"{protein_num} proteins, {cluster_num} clusters.")

15069 proteins, 14069 clusters.


In [16]:
"""Sample proteins for each cluster"""
random.seed(seed)
proteins = []

for clusterid, clusterpros in cluster_proteins.items():
    selected_pros = random.sample(clusterpros, k=cluster_max_pro_num)
    proteins.extend(selected_pros)

cur_protein_num = len(proteins)
cur_annots_num = sum([len(p.annots) for p in proteins])

print(f"{cur_protein_num} proteins reserved,",
      f"{cur_annots_num} annotaions reserved", f"({round(cur_annots_num/cur_protein_num, 2)} annots per protein).")

14069 proteins reserved, 145676 annotaions reserved (10.35 annots per protein).


### Dump Proteins

In [17]:
"""Dump proteins"""
#! Comment this block to avoid write wrong proteins 
# if not os.path.exists(out_dir):
#     os.makedirs(out_dir, exist_ok=True)

# with open(os.path.join(out_dir, "proteins.pkl"), "wb") as f:
#     pickle.dump(proteins, f)

In [5]:
"""Load pickle file"""
random.seed(seed)
with open(os.path.join(out_dir, "proteins.pkl"), "rb") as f:
    proteins = pickle.load(f)
random.shuffle(proteins)

### Step 5: Truncate Terms

In [6]:
"""Gather proteins into terms"""
term2pros = defaultdict(set)
for pro in proteins:
    for annot in pro.annots:
        goid = annot.goid
        term2pros[goid].add(pro.dbid)

In [7]:
"""Filter out terms of too few proteins"""
reserved_terms = set()
for term, pros in term2pros.items():
    if len(pros) >= min_pro_per_term:
        reserved_terms.add(term)

print(f"{len(reserved_terms)} of {len(term2pros)} ({round(len(reserved_terms)/len(term2pros)*100, 2)}%)",
      f"terms are reserved as their member proteins are more than {min_pro_per_term}")

648 of 13319 (4.87%) terms are reserved as their member proteins are more than 30


In [8]:
"""Filter out terms of too shallow"""
def read_term_depth(_dir, dep):
    # f = open(os.path.join(_dir, "term_leaf.tsv"))
    # terms = set([l.strip().split()[0] for l in f.readlines()[1:]])

    terms = set()
    f = open(os.path.join(_dir, "term_depth.tsv"))
    for l in f.readlines()[1:]:
        t, i, d = l.strip().split()
        if int(d) >= dep:
            terms.add(t)
    
    # f = open(os.path.join(_dir, "term_leaf.tsv"))
    # leaves = set([l.strip().split()[0] for l in f.readlines()[1:]])
    # terms = terms & leaves

    return terms

# bp_valid_terms = read_term(os.path.join(processed_go_dir, 'bp'))
# mf_valid_terms = read_term(os.path.join(processed_go_dir, 'mf'))
# cc_valid_terms = read_term(os.path.join(processed_go_dir, 'cc'))
bp_valid_terms = read_term_depth(os.path.join(processed_go_dir, 'bp'), min_term_depth['bp'])
mf_valid_terms = read_term_depth(os.path.join(processed_go_dir, 'mf'), min_term_depth['mf'])
cc_valid_terms = read_term_depth(os.path.join(processed_go_dir, 'cc'), min_term_depth['cc'])
valid_terms = bp_valid_terms | mf_valid_terms | cc_valid_terms

pre_terms_num = len(reserved_terms)
reserved_terms = reserved_terms & valid_terms
cur_terms_num = len(reserved_terms)

print(f"{cur_terms_num} of {pre_terms_num} ({round(cur_terms_num/pre_terms_num*100, 2)}%)",
      f"terms are reserved")

414 of 648 (63.89%) terms are reserved


In [9]:
"""Filter out terms that are ancestors of other terms"""
def read_term_ancestor(_dir, reserved_terms):
    term_ances = {}
    delete_terms = set()

    idx2term = {}
    with open(os.path.join(_dir, "term_index.tsv"), "r") as f:
        for l in f.readlines()[1:]:
            t, i = l.split("\t")
            idx2term[int(i)] = t

    with open(os.path.join(_dir, "term_ances.tsv"), "r") as f:
        for l in f.readlines()[1:]:
            t, _, ances = l.split("\t")
            if t in reserved_terms:
                for _t in [idx2term[int(i)] for i in ances.strip().split()]:
                    if _t in reserved_terms:
                        delete_terms.add(_t)
    return delete_terms

bp_del_terms = read_term_ancestor(os.path.join(processed_go_dir, 'bp'), reserved_terms)
mf_del_terms = read_term_ancestor(os.path.join(processed_go_dir, 'mf'), reserved_terms)
cc_del_terms = read_term_ancestor(os.path.join(processed_go_dir, 'cc'), reserved_terms)
del_terms = bp_del_terms | mf_del_terms | cc_del_terms

pre_terms_num = len(reserved_terms)
reserved_terms = reserved_terms - del_terms
cur_terms_num = len(reserved_terms)

print(f"{cur_terms_num} of {pre_terms_num} ({round(cur_terms_num/pre_terms_num*100, 2)}%)",
      f"terms are reserved")

317 of 414 (76.57%) terms are reserved


In [10]:
"""Remove terms"""
pre_protein_num = len(proteins)
pre_annots_num = sum([len(pro.annots) for pro in proteins])

for pro in proteins:
    pro.annots = [_annot for _annot in pro.annots if _annot.goid in reserved_terms]
# remove proteins without annotations
proteins = [pro for pro in proteins if len(pro.annots) > 0]

cur_protein_num = len(proteins)
cur_annots_num = sum([len(pro.annots) for pro in proteins])

print(f"{cur_protein_num} of {pre_protein_num} proteins reserved,",
    f"{cur_annots_num} of {pre_annots_num} annotaions reserved", f"({round(cur_annots_num/cur_protein_num, 2)} annots per protein).")

10985 of 14069 proteins reserved, 30922 of 145676 annotaions reserved (2.81 annots per protein).


### Store Protein Data

In [11]:
def write_file(fp, lines, header=None):
    with open(fp, "w") as f:
        if header is not None:
            f.write(header)
        f.writelines(lines)

In [12]:
"""Store seqs and coords"""
pro_seqs = [f"{pro.dbid}\t{pro.sequence}\n" for pro in proteins]
write_file(os.path.join(out_dir, "seqs.tsv"), pro_seqs)

# store seq,pdb csv for structure embedding with pre-trained model
pro_seq_pdbs = [f"{pro.dbid},{pro.sequence},{pro.dbid}.pdb\n" for pro in proteins]
write_file(os.path.join(out_dir, "seq_pdbs.csv"), pro_seq_pdbs, header="name,sequence,pdb\n")


### Split BP MF CC

In [13]:
def split_terms(processed_cat_dir, tgt_dir):
    os.makedirs(tgt_dir, exist_ok=True)

    """Read go terms"""
    term2idx = {}
    idx2term = {}
    with open(os.path.join(processed_cat_dir, "term_index.tsv"), "r") as f:
        for l in f.readlines()[1:]:
            t, i = l.split("\t")
            term2idx[t] = int(i)
            idx2term[int(i)] = t

    pro_terms = defaultdict(set)
    term_pros = defaultdict(set)

    for pro in proteins:
        terms = [a.goid for a in pro.annots if a.goid in term2idx]
        if len(terms) == 0:
            continue

        pro_terms[pro.dbid] = set(terms)
        for t in terms:
            term_pros[t].add(pro.dbid)
        
    return pro_terms, term_pros

bp_pro_terms, bp_term_pros = split_terms(os.path.join(processed_go_dir, "bp"), os.path.join(out_dir, "bp"))
mf_pro_terms, mf_term_pros = split_terms(os.path.join(processed_go_dir, "mf"), os.path.join(out_dir, "mf"))
cc_pro_terms, cc_term_pros = split_terms(os.path.join(processed_go_dir, "cc"), os.path.join(out_dir, "cc"))

In [14]:
print(len(bp_pro_terms), len(bp_term_pros))
print(len(mf_pro_terms), len(mf_term_pros))
print(len(cc_pro_terms), len(cc_term_pros))

4944 156
4909 72
8834 89


### Split Data


In [22]:

random.seed(9)

def split_data(pro_terms, term_pros, processed_cat_dir):
    terms = list(term_pros.keys())
    proteins = list(pro_terms.keys())

    random.shuffle(terms)
    random.shuffle(proteins)

    unseen_proteins = set(random.sample(proteins, k=int(len(proteins) * 0.1)))
    unseen_terms = set(random.sample(terms, k=int(len(terms) * 0.1)))

    samples = [(p, t) for p in pro_terms for t in pro_terms[p]]
    train_samples = []
    test_samples =[]

    # 1:  seen proteins, unseen terms
    # 2:  unseen proteins, seen terms
    # 3:  unseen proteins, unseen terms        
    for p, t in samples:
        if t in unseen_terms or p in unseen_proteins:
            test_samples.append((p, t))    
        else:
            train_samples.append((p, t))

    train_proteins = set([p for p, t in train_samples])
    test_proteins = set([p for p, t in test_samples])
    train_terms = set([t for p, t in train_samples])
    test_terms = set([t for p, t in test_samples])

    for i, (p, t) in enumerate(test_samples):
        if p in train_proteins and t in train_terms:
            assert ValueError("Cannot protein and term are both seen")
        elif p in train_proteins and t not in train_terms:
            test_samples[i] = (p, t, 1)
        elif p not in train_proteins and t in train_terms:
            test_samples[i] = (p, t, 2)
        elif p not in train_proteins and t not in train_terms:
            test_samples[i] = (p, t, 3)

    print("Pos of Train:", len(train_samples), "\tPos of Test:", len(test_samples))
    print(
        f"1: {len([i for p, t, i in test_samples if i == 1])}"
        f"\n2: {len([i for p, t, i in test_samples if i == 2])}"
        f"\n3: {len([i for p, t, i in test_samples if i == 3])}")
    write_file(os.path.join(processed_cat_dir, "train_pro.tsv"), [p+"\n" for p in train_proteins])
    write_file(os.path.join(processed_cat_dir, "train_term.tsv"), [t+"\n" for t in train_terms])
    write_file(os.path.join(processed_cat_dir, "test_pro.tsv"), [p+"\n" for p in test_proteins])
    write_file(os.path.join(processed_cat_dir, "test_term.tsv"), [t+"\n" for t in test_terms])
    write_file(os.path.join(processed_cat_dir, "train.tsv"), [p+'\t'+t+"\n" for p, t in train_samples], header="Protein\tTerm\n")
    write_file(os.path.join(processed_cat_dir, "test.tsv"), [p+'\t'+t+"\t"+str(i)+"\n" for p, t, i in test_samples],header="Protein\tTerm\tType\n")

    print("pro: ", len(train_proteins), len(test_proteins), len(train_proteins&test_proteins))
    print("term: ", len(train_terms), len(test_terms), len(train_terms&test_terms))
    print()

split_data(bp_pro_terms, bp_term_pros, os.path.join(out_dir, "bp"))
split_data(mf_pro_terms, mf_term_pros, os.path.join(out_dir, "mf"))
split_data(cc_pro_terms, cc_term_pros, os.path.join(out_dir, "cc"))

Pos of Train: 6905 	Pos of Test: 1433
1: 306
2: 731
3: 396
pro:  4128 1102 286
term:  141 155 140

Pos of Train: 5765 	Pos of Test: 1195
1: 229
2: 691
3: 275
pro:  4194 941 226
term:  65 72 65

Pos of Train: 12713 	Pos of Test: 2911
1: 789
2: 1377
3: 745
pro:  7383 2178 727
term:  81 88 80

